In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from fairlearn.metrics import MetricFrame
from sklearn import metrics as skm
import seaborn as sns
import os.path as path
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os, csv, shapely, shapely.wkt, descartes, geopy, pickle, cloudpickle

pd.set_option('display.max_rows', 10)
sns.set()
sns.color_palette("ch:s=.25,rot=-.25")
np.set_printoptions(suppress=True)
# pd.options.display.float_format = '{:.2f}'.format


Bad key "text.kerning_factor" on line 4 in
/Users/yhannnn/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [51]:
NUM_STATIONS_INC = 2

In [52]:
# NEED MODIFY
EPOCH = 50
LAMBDA = 0.1
FOLDER = 'fairst_all_models/e_50_lambda_0_1'
PRED_FILE_NAME = "inference_stations_2_fused_model_pred_0.1_1641294380.6203291.csv"

In [53]:
CITY = "C_"
save_dir = CITY + 'reg_res/'
save_dir_preprocess = os.path.join(save_dir, CITY + 'preprocess_fig/')
save_dir_models = os.path.join(save_dir, CITY + 'model_res/') 
save_dir_forecast = os.path.join(save_dir, CITY + 'forecast/')
save_dir_fairst = os.path.join(save_dir, CITY + 'fairst/')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
if not os.path.exists(save_dir_preprocess):
    os.makedirs(save_dir_preprocess)

if not os.path.exists(save_dir_models):
    os.makedirs(save_dir_models)
    
if not os.path.exists(save_dir_forecast):
    os.makedirs(save_dir_forecast)
    
if not os.path.exists(save_dir_fairst):
    os.makedirs(save_dir_fairst)

In [54]:
pred_df = pd.read_csv(os.path.join(save_dir_fairst, FOLDER, PRED_FILE_NAME), index_col = 0)
gt_df = pd.read_csv(os.path.join(save_dir_fairst, "test_df_all.csv"), index_col = 0)
pos_info = pd.read_csv(os.path.join(save_dir_fairst, "df_demo.csv"), index_col = 0)
income_info = pd.read_csv(os.path.join(save_dir_fairst, "df_income.csv"), index_col = 0)
income_info = income_info.groupby(by = "cell_num").mean().reset_index()
income_info.income_group = income_info.income_group.astype(int)
demo_raw = pd.read_csv(os.path.join(save_dir_fairst, "demo_raw_all.csv"))
pred_df.index = pd.to_datetime(pred_df.index)
gt_df.index = pd.to_datetime(gt_df.index)
gt_df = gt_df[gt_df.index.isin(pred_df.index)]
normalized_pop_df = pd.read_csv(os.path.join(save_dir_fairst, "normalized_pop_df.csv"), index_col = 0)

In [55]:
def df_transform(df):
    pos_l = df.columns
    df_trans = pd.DataFrame(columns = ["pos", "outflow"])

    for i in np.arange(len(pos_l)):
        pos = pos_l[i]
        temp = df[pos].to_frame()
        temp["pos"] = [temp.columns.values[0]]*len(temp)
        temp.rename(columns = {pos: "outflow"}, inplace = True)
        df_trans = df_trans.append(temp)
        
    return df_trans

In [56]:
pred_df_month = pred_df.groupby(pred_df.index.month).sum()

In [57]:
pred_df_month

,2_0,2_1,3_0,3_1,3_2,4_0,4_1,5_0,5_1,5_2,...,86_22,87_21,87_22,88_22,89_22,89_23,89_24,90_24,90_25,91_26
2,25.425518,0.0,3.099336,0.0,0.0,10.111310,0.0,10.111311,0.0,0.0,...,0.0,0.0,0.169659,12.391787,0.0,0.0,0.0,65.425639,11.262430,0.0
3,0.628851,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.550529,0.0,0.0,0.0,65.980471,0.000000,0.0
4,18.128517,0.0,0.168264,0.0,0.0,4.342311,0.0,4.342313,0.0,0.0,...,0.0,0.0,0.000000,5.895980,0.0,0.0,0.0,58.030031,4.742054,0.0
5,1.416602,0.0,0.000000,0.0,0.0,0.000243,0.0,0.000244,0.0,0.0,...,0.0,0.0,0.000000,0.831895,0.0,0.0,0.0,53.854908,0.178837,0.0
6,28.155509,0.0,3.307993,0.0,0.0,10.622283,0.0,10.622288,0.0,0.0,...,0.0,0.0,0.397060,13.935612,0.0,0.0,0.0,71.316017,9.399871,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,18.743105,0.0,0.574738,0.0,0.0,6.324804,0.0,6.324804,0.0,0.0,...,0.0,0.0,0.000000,5.417077,0.0,0.0,0.0,67.160807,4.834520,0.0
9,1.494163,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,57.699823,0.000000,0.0
10,6.897343,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,3.590138,0.0,0.0,0.0,49.978828,0.034025,0.0
11,6.656415,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,5.136502,0.0,0.0,0.0,35.452575,0.000000,0.0


In [58]:
df_transform_m = df_transform(pred_df_month).groupby('pos').sum()
df_transform_m = df_transform_m.reset_index()

In [59]:
# df_pred_trans = df_transform(pred_df_month)
pred_final = df_transform_m.merge(pos_info)

pred_final_m = pred_final.merge(income_info, on = 'cell_num')

In [60]:
pred_final_m.to_csv(os.path.join(save_dir_fairst, "fairst_forecast_inc_{}.csv".format(NUM_STATIONS_INC)))